In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [14]:
# Dataset Path
data_dir = "E:/FD_ERS_TEAM"
img_size = 48
batch_size = 64

In [15]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # Random rotation
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Random zoom
    brightness_range=[0.8, 1.2],  # Random brightness adjustment
    horizontal_flip=True,  # Random horizontal flip
    validation_split=0.2  # 20% of data for validation
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 28508 images belonging to 2 classes.
Found 7126 images belonging to 2 classes.


In [16]:
# 🔥 LeNet-Based Model
model = Sequential([
    # Layer 1: Conv + Pool
    Conv2D(6, (5, 5), activation='relu', padding='same', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    # Layer 2: Conv + Pool
    Conv2D(16, (5, 5), activation='relu', padding='valid'),
    MaxPooling2D(pool_size=(2, 2)),

    # Flatten
    Flatten(),

    # Fully Connected Layers
    Dense(120, activation='relu'),
    Dropout(0.5),  # Add dropout to prevent overfitting
    Dense(84, activation='relu'),
    Dropout(0.5),  # Add dropout to prevent overfitting
    Dense(train_generator.num_classes, activation='softmax')
])

In [17]:
# 🚀 Optimized AdamW Optimizer
optimizer = AdamW(learning_rate=0.001, weight_decay=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# 🔧 Callbacks
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, min_lr=1e-6, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("best_model_lenet.h5", monitor='val_accuracy', save_best_only=True, mode='max')

In [18]:
# 🚀 Train Model
epochs = 25  # Increase epochs if needed
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[lr_reduction, early_stopping, model_checkpoint]
)

Epoch 1/25
446/446 [==============================] - 428s 956ms/step - loss: 0.5057 - accuracy: 0.8051 - val_loss: 0.4950 - val_accuracy: 0.8056 - lr: 0.0010
Epoch 2/25
446/446 [==============================] - 54s 120ms/step - loss: 0.4972 - accuracy: 0.8057 - val_loss: 0.4927 - val_accuracy: 0.8056 - lr: 0.0010
Epoch 3/25
446/446 [==============================] - 57s 128ms/step - loss: 0.4949 - accuracy: 0.8057 - val_loss: 0.4930 - val_accuracy: 0.8056 - lr: 0.0010
Epoch 4/25
446/446 [==============================] - 54s 122ms/step - loss: 0.4948 - accuracy: 0.8057 - val_loss: 0.4926 - val_accuracy: 0.8056 - lr: 0.0010
Epoch 5/25
446/446 [==============================] - 54s 121ms/step - loss: 0.4951 - accuracy: 0.8057 - val_loss: 0.4925 - val_accuracy: 0.8056 - lr: 0.0010
Epoch 6/25
446/446 [==============================] - 58s 131ms/step - loss: 0.4941 - accuracy: 0.8057 - val_loss: 0.4929 - val_accuracy: 0.8056 - lr: 0.0010
Epoch 7/25
446/446 [==============================]

KeyboardInterrupt: 